In [ ]:
!pip install --upgrade keras
import os
os.environ["KERAS_BACKEND"] = "jax"
import keras
import numpy as np
import cv2

class DepthToSpace(keras.layers.Layer):
    def __init__(self, block_size):
        super().__init__()
        self.block_size = block_size

    def call(self, input):
        batch, height, width, depth = keras.ops.shape(input)
        depth = depth // (self.block_size**2)
        x = keras.ops.reshape(input, [batch, height, width, self.block_size, self.block_size, depth])
        x = keras.ops.transpose(x, [0, 1, 3, 2, 4, 5])
        x = keras.ops.reshape(x, [batch, height * self.block_size, width * self.block_size, depth])
        return x

# Settings
filters = 64

# Build the model:
inputs = keras.layers.Input(shape=(None,None,1))
conv0 = keras.layers.Conv2D(filters=filters, kernel_size=3, padding='same')(inputs)

# Internal convolutions
conv1 = keras.layers.Conv2D(filters=filters, kernel_size=3, padding='same', activation='relu')(conv0)
conv2 = keras.layers.Conv2D(filters=filters, kernel_size=3, padding='same', activation='relu')(conv1)
conv3 = keras.layers.Conv2D(filters=filters, kernel_size=3, padding='same', activation='relu')(conv2)
conv4 = keras.layers.Conv2D(filters=filters, kernel_size=3, padding='same', activation='relu')(conv3)

# Feature Fusion
mix_global = keras.layers.Conv2D(filters=filters, kernel_size=3, padding='same')(conv4)
add_global = keras.layers.Add()([mix_global, conv0])

# Upsampler
features = keras.layers.Conv2D(filters=4, kernel_size=3, padding='same')(add_global)
outputs = DepthToSpace(2)(features)

# Defining the model
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile the model.
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=keras.losses.MeanAbsoluteError())
model.summary()
#keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.load_weights("/content/model.h5", skip_mismatch=True, by_name=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/Datasets/Manga109_Train_LR.zip /content/LR.zip
!cp /content/drive/MyDrive/Datasets/Manga109_Train_HR.zip /content/HR.zip

!unzip /content/LR.zip
!unzip /content/HR.zip

In [ ]:
!rm -rf /content/HR
!rm -rf /content/LR

In [ ]:
import glob
import cv2

filelist1 = sorted(glob.glob('/content/LR/*'))
train_in = []
for myFile in filelist1:
    image = cv2.imread(myFile, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, 0)
    train_in.append(image)
train_in = np.array(train_in).astype(np.float32) / 255.0
train_in = np.clip(train_in, 0.0, 1.0)
train_in = np.expand_dims(train_in, axis=-1)

filelist2 = sorted(glob.glob('/content/HR/*'))
train_ref = []
for myFile in filelist2:
    image = cv2.imread(myFile, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, 0)
    train_ref.append(image)
train_ref = np.array(train_ref).astype(np.float32) / 255.0
train_ref = np.clip(train_ref, 0.0, 1.0)
train_ref = np.expand_dims(train_ref, axis=-1)

In [ ]:
# Train the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=keras.losses.MeanAbsoluteError())
history = model.fit(train_in, train_ref, epochs=500, batch_size=64, verbose=1)

In [ ]:
# Save the weights
model.save('/content/model.h5')
model.save('/content/model.keras')

In [ ]:
# Make a single prediction
input = cv2.imread('/content/downscaled.png', cv2.IMREAD_GRAYSCALE)
input = np.array(input).astype(np.float32) / 255.0
input = np.clip(input, 0, 1)
input = np.expand_dims(input, axis=0)
input = np.expand_dims(input, axis=-1)

pred = model.predict(input)
pred = np.clip(pred, 0, 1)
pred = np.squeeze(pred)
pred = pred * 255
pred = np.squeeze((np.around(pred)).astype(np.uint8))

cv2.imwrite('/content/prediction.png', pred)

In [ ]:
#Generate normal shader
def generate_shader_code(current_layer, previous_layer, channels_in, channels_out):
    passes_in = int(np.ceil(channels_in / 4))
    passes_out = int(np.ceil(channels_out / 4))

    if previous_layer.name == "input_layer":
        previous_layer.name = "LUMA"

    shader_code = ""
    for pass_idx in range(passes_out):
        if "conv2d" in current_layer.name:
            shader_code += f"//!DESC ArtCNN C4F{filters} ({current_layer.name.title().replace('_', '-')}-{pass_idx})\n"
        else:
            shader_code += f"//!DESC ArtCNN C4F{filters} ({current_layer.name.title().replace('_', '-')})\n"

        shader_code += f"//!HOOK LUMA\n"

        if previous_layer.name == "LUMA":
            shader_code += f"//!BIND {previous_layer.name}\n"
        elif "add" in previous_layer.name:
            for i in range(passes_in):
                shader_code += f"//!BIND conv2d_{i}\n"
                shader_code += f"//!BIND conv2d_5_{i}\n"
        elif "conv2d" in current_layer.name:
            for i in range(passes_in):
                shader_code += f"//!BIND {previous_layer.name}_{i}\n"
        elif "depth" in current_layer.name:
            shader_code += f"//!BIND {previous_layer.name}_0\n"

        if "depth" in current_layer.name:
            shader_code += f"//!WIDTH LUMA.w 2.0 *\n"
            shader_code += f"//!HEIGHT LUMA.h 2.0 *\n"
        else:
            shader_code += f"//!SAVE {current_layer.name}_{pass_idx}\n"
            shader_code += f"//!WIDTH LUMA.w\n"
            shader_code += f"//!HEIGHT LUMA.h\n"

        shader_code += f"//!COMPONENTS 4\n"
        shader_code += f"//!WHEN OUTPUT.w LUMA.w / 1.3 > OUTPUT.h LUMA.h / 1.3 > *\n\n"
        shader_code += "vec4 hook() {\n"

        if "conv2d" in current_layer.name:
            biases = current_layer.get_weights()[1][pass_idx*4:(pass_idx+1)*4]
            biases_str = ", ".join(str(w) for w in biases.flatten())
            shader_code += f"    vec4 result = vec4({biases_str});\n"

            for z in range(passes_in):
                for y in range(-1, 2):
                    for x in range(-1, 2):
                        weights = current_layer.get_weights()[0][y+1, x+1, z*4:(z+1)*4, pass_idx*4:(pass_idx+1)*4]
                        weights_str = ", ".join(str(w) for w in weights.flatten())

                        if weights_str:
                            if previous_layer.name == "LUMA":
                                shader_code += f"    result += vec4({weights_str}) * {previous_layer.name}_texOff(vec2({x}, {y})).x;\n"
                            elif "add" in previous_layer.name:
                                shader_code += f"    result += mat4({weights_str}) * (conv2d_5_{z}_texOff(vec2({x}, {y})) + conv2d_{z}_texOff(vec2({x}, {y})));\n"
                            else:
                                shader_code += f"    result += mat4({weights_str}) * {previous_layer.name}_{z}_texOff(vec2({x}, {y}));\n"

            if any(layer_name in current_layer.name for layer_name in ["conv2d_1", "conv2d_2", "conv2d_3", "conv2d_4"]):
                shader_code += "    return max(result, vec4(0.0));\n"
            else:
                shader_code += "    return result;\n"

        elif "depth" in current_layer.name:
            shader_code += f"    vec4 result = vec4(0.0, 0.0, 0.0, 1.0);\n"
            shader_code += f"    vec2 f0 = fract({previous_layer.name}_0_pos * {previous_layer.name}_0_size);\n"
            shader_code += f"    ivec2 i0 = ivec2(f0 * vec2(2.0));\n"
            shader_code += f"    result.x = {previous_layer.name}_0_tex((vec2(0.5) - f0) * {previous_layer.name}_0_pt + {previous_layer.name}_0_pos)[i0.y * 2 + i0.x];\n"
            shader_code += f"    return clamp(result, 0.0, 1.0);\n"

        shader_code += "}\n\n"
    return shader_code

################################################################################
filters = model.layers[1].filters
shader_code = """// MIT License

// Copyright (c) 2024 Joao Chrisostomo

// Permission is hereby granted, free of charge, to any person obtaining a copy
// of this software and associated documentation files (the "Software"), to deal
// in the Software without restriction, including without limitation the rights
// to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
// copies of the Software, and to permit persons to whom the Software is
// furnished to do so, subject to the following conditions:

// The above copyright notice and this permission notice shall be included in all
// copies or substantial portions of the Software.

// THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
// IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
// FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
// AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
// LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
// OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
// SOFTWARE.

"""
for i in range(1, len(model.layers)):
    if "input" in model.layers[i - 1].name:
        shader_code += generate_shader_code(model.layers[i], model.layers[i - 1], 1, filters)
    elif "conv2d_6" in model.layers[i].name:
        shader_code += generate_shader_code(model.layers[i], model.layers[i - 1], filters, 4)
    elif "conv2d" in model.layers[i].name:
        shader_code += generate_shader_code(model.layers[i], model.layers[i - 1], filters, filters)
    elif "depth_to_space" in model.layers[i].name:
        shader_code += generate_shader_code(model.layers[i], model.layers[i - 1], 4, 1)

print(shader_code)
with open("meme.glsl", mode="w") as f:
    f.write(shader_code)

In [ ]:
#Generate computer shader
def generate_shader_code(current_layer, previous_layer, channels_in, channels_out):
    passes_in = int(np.ceil(channels_in / 4))
    passes_out = int(np.ceil(channels_out / 4))

    if previous_layer.name == "input_layer":
        previous_layer.name = "LUMA"

    shader_code = ""
    for pass_idx in range(passes_out):
        if "conv2d" in current_layer.name:
            shader_code += f"//!DESC ArtCNN C4F{filters} ({current_layer.name.title().replace('_', '-')}-{pass_idx})\n"
        else:
            shader_code += f"//!DESC ArtCNN C4F{filters} ({current_layer.name.title().replace('_', '-')})\n"

        shader_code += f"//!HOOK LUMA\n"

        if previous_layer.name == "LUMA":
            shader_code += f"//!BIND {previous_layer.name}\n"
        elif "add" in previous_layer.name:
            for i in range(passes_in):
                shader_code += f"//!BIND conv2d_{i}\n"
                shader_code += f"//!BIND conv2d_5_{i}\n"
        elif "conv2d" in current_layer.name:
            for i in range(passes_in):
                shader_code += f"//!BIND {previous_layer.name}_{i}\n"
        elif "depth" in current_layer.name:
            shader_code += f"//!BIND {previous_layer.name}_0\n"

        if "depth" in current_layer.name:
            shader_code += f"//!WIDTH LUMA.w 2.0 *\n"
            shader_code += f"//!HEIGHT LUMA.h 2.0 *\n"
        else:
            shader_code += f"//!SAVE {current_layer.name}_{pass_idx}\n"
            shader_code += f"//!WIDTH LUMA.w\n"
            shader_code += f"//!HEIGHT LUMA.h\n"

        shader_code += f"//!COMPONENTS 4\n"
        shader_code += f"//!COMPUTE 32 32\n"
        shader_code += f"//!WHEN OUTPUT.w LUMA.w / 1.3 > OUTPUT.h LUMA.h / 1.3 > *\n\n"
        shader_code += "void hook() {\n"

        if "conv2d" in current_layer.name:
            biases = current_layer.get_weights()[1][pass_idx*4:(pass_idx+1)*4]
            biases_str = ", ".join(str(w) for w in biases.flatten())
            shader_code += f"    vec4 result = vec4({biases_str});\n"

            for z in range(passes_in):
                for y in range(-1, 2):
                    for x in range(-1, 2):
                        weights = current_layer.get_weights()[0][y+1, x+1, z*4:(z+1)*4, pass_idx*4:(pass_idx+1)*4]
                        weights_str = ", ".join(str(w) for w in weights.flatten())

                        if weights_str:
                            if previous_layer.name == "LUMA":
                                shader_code += f"    result += vec4({weights_str}) * {previous_layer.name}_texOff(vec2({x}, {y})).x;\n"
                            elif "add" in previous_layer.name:
                                shader_code += f"    result += mat4({weights_str}) * (conv2d_5_{z}_texOff(vec2({x}, {y})) + conv2d_{z}_texOff(vec2({x}, {y})));\n"
                            else:
                                shader_code += f"    result += mat4({weights_str}) * {previous_layer.name}_{z}_texOff(vec2({x}, {y}));\n"

            if any(layer_name in current_layer.name for layer_name in ["conv2d_1", "conv2d_2", "conv2d_3", "conv2d_4"]):
                shader_code += "    imageStore(out_image, ivec2(gl_GlobalInvocationID), max(result, vec4(0.0)));\n"
            else:
                shader_code += "    imageStore(out_image, ivec2(gl_GlobalInvocationID), result);\n"

        elif "depth" in current_layer.name:
            shader_code += f"    vec4 result = vec4(0.0, 0.0, 0.0, 1.0);\n"
            shader_code += f"    vec2 f0 = fract({previous_layer.name}_0_pos * {previous_layer.name}_0_size);\n"
            shader_code += f"    ivec2 i0 = ivec2(f0 * vec2(2.0));\n"
            shader_code += f"    result.x = {previous_layer.name}_0_tex((vec2(0.5) - f0) * {previous_layer.name}_0_pt + {previous_layer.name}_0_pos)[i0.y * 2 + i0.x];\n"
            shader_code += f"    imageStore(out_image, ivec2(gl_GlobalInvocationID), clamp(result, 0.0, 1.0));\n"

        shader_code += "}\n\n"
    return shader_code

################################################################################
filters = model.layers[1].filters
shader_code = """// MIT License

// Copyright (c) 2024 Joao Chrisostomo

// Permission is hereby granted, free of charge, to any person obtaining a copy
// of this software and associated documentation files (the "Software"), to deal
// in the Software without restriction, including without limitation the rights
// to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
// copies of the Software, and to permit persons to whom the Software is
// furnished to do so, subject to the following conditions:

// The above copyright notice and this permission notice shall be included in all
// copies or substantial portions of the Software.

// THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
// IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
// FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
// AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
// LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
// OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
// SOFTWARE.

"""
for i in range(1, len(model.layers)):
    if "input" in model.layers[i - 1].name:
        shader_code += generate_shader_code(model.layers[i], model.layers[i - 1], 1, filters)
    elif "conv2d_6" in model.layers[i].name:
        shader_code += generate_shader_code(model.layers[i], model.layers[i - 1], filters, 4)
    elif "conv2d" in model.layers[i].name:
        shader_code += generate_shader_code(model.layers[i], model.layers[i - 1], filters, filters)
    elif "depth_to_space" in model.layers[i].name:
        shader_code += generate_shader_code(model.layers[i], model.layers[i - 1], 4, 1)

print(shader_code)
with open("meme.glsl", mode="w") as f:
    f.write(shader_code)